In [5]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [90]:
total_players =purchase_data["SN"].nunique()
# purchase_data["SN"].nunique()
print(f"Player count is {total_players}")
print(total_players)

Player count is 576
576


In [22]:
unique_items = len(purchase_data["Item ID"].unique())
avg_price = purchase_data["Price"].mean()
num_purchases = purchase_data["SN"].count()
tot_rev = purchase_data["Price"].sum()
print(unique_items)
print(avg_price)
print(num_purchases)
print(tot_rev)

179
3.050987179487176
780
2379.77


In [29]:
# Place all calculated data in a summary DF
summary_df = pd.DataFrame({"Number of Unique Items": [unique_items], 
                           "Average Price": avg_price, 
                           "Number of Purchases": num_purchases,"Total Revenue": tot_rev})
summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [116]:
# calculate percentage and count of gender
gender = purchase_data.groupby("Gender")["SN"].nunique()
gender = gender.to_frame()
gender.rename(columns = {"SN": "Total Count"})
# female = gender[0]
# male = gender[1]
# other = gender[2]
gender
# print(female)
# print(male)


,SN
Gender,
Female,81
Male,484
Other / Non-Disclosed,11


In [72]:
# calculate percentage and count of gender
# print(purchase_data["Gender"].value_counts())
# purchase_data["Gender"].value_counts()
unique_players = purchase_data["Gender"].value_counts()
print(unique_players)


Male                     652
Female                   113
Other / Non-Disclosed     15
Name: Gender, dtype: int64
